## Projeto Encontrar Rios

Você irá receber uma lista bi-dimensional com altura e largura não necessáriamente iguais contendo apenas 0's e 1's. Cada 1 representa um pedaço de rio, enquanto os 0's representam terra. Rios são compostos por 1's adjacentes horizontalmente ou verticalmente (mas não diagonalmente). O número de 1's adjacentes representa o tamanho do rio.

Note que o rio pode fazer curvas, isto é, rios podem ter formato de L ou até mesmo de cruz e são considerados um rio só.

Crie um algoritmo que receba esta lista bi-dimensional e retorne uma lista com os tamanhos dos rios dentro dessa estrutura, os tamanhos de rios dentro da lista resposta não precisam ter uma ordem específica.

**Exemplo de entrada:**

```python
matrix = [
    [1, 0, 0, 1, 0],
    [1, 0, 1, 0, 0],
    [0, 0, 1, 0, 1],
    [1, 0, 1, 0, 1],
    [1, 0, 1, 1, 0],
]
```

**Saída esperada:**

```python
[1, 2, 2, 2, 5] # Note que os números poderiam estar em qualquer ordem dentro da lista
```

podemos visualizar os rios claramente na seguinte estrutura:

```python
[1,  ,  , 1,  ]
[1,  , 1,  ,  ]
[ ,  , 1,  , 1]
[1,  , 1,  , 1]
[1,  , 1, 1,  ]
```

## Descrição da solução:

É importante explicar a ideia que tive para resolver o programa. Depois da primeira aula sobre grafos, na qual se mostrou a rotina de busca em profundidade, me dediquei a procurar uma solução na qual poderia resolver por meio desta rotina. Abaixo segue uma descrição passo a passo da minha solução. Inicialmente, foi tudo realizado "manualmente".

O primeiro passo foi escrever um grafo que representasse matrix:

```python
matrix = [
[1,  ,  , 1,  ]
[1,  , 1,  ,  ]
[ ,  , 1,  , 1]
[1,  , 1,  , 1]
[1,  , 1, 1,  ]
].
```
Pensei em substituir cada posição na matrix por valores numéricos de elementos de um grafo:
```python
matrix = [
[ 0,   ,   ,  1,   ], 
[ 2,   ,  3,   ,   ], 
[  ,   ,  4,   ,  5], 
[ 6,   ,  7,   ,  8], 
[ 9,   , 10, 11,   ], 
].
```
Torna-se natural transformar matrix, agora, em uma lista de adjacências:

```python
grafo = [
0: [2]
1: []
2: [0]
3: [4]
4: [3, 7]
5: [8]
6: [9]
7: [4, 10]
8: [5]
9: [6]
10: [7, 11]
11: [10]
];
```
Observação: na notação 0: [2], o 0: foi colocado somente para facilitar o entendimento de qual nó está sendo analisado na lista de adjacência que é uma lista de listas.


Finalmente, modifiquei a busca em profundidade para determinar o caminho neste grafo a partir de um vértice inicial. Por exemplo, para o vértice 7, consegue-se o seguinte resultado:

```python
[7, 4, 3, 10, 11].
```

Porém, se fizer a busca em profundidade para o vértice 3, temos o mesmo resultado, pois o vértice 3 pertence ao mesmo caminho:

```python
[3, 4, 7, 10, 11].
```
Portanto, teria-se de fazer uma rotina que somente chamasse os vértices que ainda não pertenciam a um caminho anterior. A função tamanho_caminhos(grafo) realiza esta tarefa, ela retorna uma lista com o tamanho de todos os caminhos e outra lista com as listas de cada caminho.

O desafio, agora, seria fazer uma função que transformasse a matrix de uns e zeros no grafo, pois eu já sabia, possuindo o grafo, o algoritmo de busca em profundidade determinava os caminhos e tamanhos dos caminhos. Na função fazer_grafo(matrix, modo), este processo é realizado. O parâmetro modo é para definir se deseja fazer um grafo com os rios, modo = 1, ou com as ilhas, modo = 0.

Abaixo, seguem as funções criadas para conseguir a solução. No final, deixei a função mostrar_matrix_grafo, mostrar_grafo que permite visualizar os resultados com maior clareza.

In [1]:
# Define-se matrix:

matrix = [
    [1, 0, 0, 1, 0],
    [1, 0, 1, 0, 0],
    [0, 0, 1, 0, 1],
    [1, 0, 1, 0, 1],
    [1, 0, 1, 1, 0],
]

In [2]:
# Função para transformar uma matriz de zeros e uns em um grafo:

def fazer_grafo(m, modo):
    
    # m: matriz que se deseja transformar em grafo
    # modo: 1: procurar por rios ou 0: procurar ilhas

    # Verifica se foi escolhido um valor correto para modo:
    if modo not in [0, 1]:
        raise ValueError(f'fazer_grafo - Erro: modo somente pode ser 0 ou 1 - modo = {modo}')
        
    #Define o tamanho da matriz m:
    largura = len(m)
    if largura!= 0:
        altura = len(m[0])
    
    # Cria dicionário com os indices do elementos iguais a um(1) e posição do elemento na matrix:
    elemento = 0
    dicio = {}
    for i in range(largura):
        for j in range(altura):
            if m[i][j] == modo: 
                dicio[elemento] = [i, j]
                elemento += 1

    # Cria o grafo vazio:
    grafo = []
    for i in range(len(dicio)):
        grafo.append([])
        
    # Adiciona os valores do grafo:
    lista_valores = list(dicio.values())
    
    # A varredura é feita somente para baixo e para direita porque o grafo é simétrico:
    for indice in dicio.keys():
        item = [dicio[indice][0] + 1, dicio[indice][1]]
        if item in lista_valores:
            novo_indice = lista_valores.index(item)
            grafo[indice].append(novo_indice)
            # Simétrico:
            grafo[novo_indice].append(indice)
            
        item = [dicio[indice][0], dicio[indice][1] + 1]
        if item in lista_valores:
            novo_indice = lista_valores.index(item)
            grafo[indice].append(novo_indice)
            # Simétrico:
            grafo[novo_indice].append(indice)
            
    return grafo

In [3]:
# Função para fazer a busca em profundidade no grafo:

def buscaProfundidade(grafo, vertice, visitados, tamanho = 0):
            
    visitados.append(vertice)
    tamanho += 1
    
    for vizinho in grafo[vertice]:
        
        if vizinho not in visitados:
            tamanho, visitados = buscaProfundidade(grafo, vizinho, visitados, tamanho)
            
    return tamanho, visitados

In [4]:
# Função que determina o tamanho dos caminhos de um grafo:

def tamanho_caminhos(grafo):
    lista_caminhos = []
    lista_tamanhos = []
    visitados = []
    
    # Varre todo o grafo:
    for indice in range(len(grafo)):
        #Somente os vértices ainda não visitados:
        if indice not in visitados:
            # Realiza a busca em profundidade:
            tamanho, caminho = buscaProfundidade(grafo, indice, [])
            # Atualiza as listas:
            lista_tamanhos.append(tamanho)
            lista_caminhos.append(caminho)
            visitados = visitados + caminho
    # Retorna as listas:
    return lista_tamanhos, lista_caminhos

In [11]:
# Menu para determinar determinar o modo de "rios" ou "ilhas" e apresentar a lista de tamanhos:

# Escolhe o mode de rios ou ilhas:
modo = input('Digite o modo de procura (1 para rios e 0 para ilhas): ')
if modo not in ['0', '1']:
    modo = input('Digite um valor correto para o modo de procura (1 para rios e 0 para ilhas): ')
modo = int(modo)

# Transforma matrix no grafo:
grafo = fazer_grafo(matrix, modo)

# Encontram-se os tamanhos e caminhos: 
tamanhos, caminhos = tamanho_caminhos(grafo)

# Imprime os resultados em ordem crescente:
print(sorted(tamanhos))


Digite o modo de procura (1 para rios e 0 para ilhas): 1
[1, 2, 2, 2, 5]


In [6]:
# Funçao que retorna a matriz com os elementos numerados de um grafo:

def mostra_matrix_grafo(m, modo, grafo_matriz = 1):
    # m = matriz
    # modo = 1 ou 0 (rios ou ilhas)
    # grafo_matriz = 1 ou 0 (matriz tipo grafo ou matriz somente com uns)
    
    # Determina a largura e altura da matriz:
    largura = len(matrix)
    if largura != 0:
        altura = len(matrix[0])

    # Determina se vai mostrar a matrix ou matrix_grafo:
    if grafo_matriz == 1: elemento = 0
    else: elemento = 1
    
    # Cria uma string com a matriz desejada:
    string_matrix = 'matrix = [\n'
    for i in range(largura):
        
        string_matrix += '['
        for j in range(altura):
            
            if j != altura - 1:
                fim = ', '
            else:
                fim = '], '
                
            if matrix[i][j] == modo:
                if elemento <= 9:
                    string_matrix += ' ' + str(elemento) + fim
                else:
                    string_matrix += str(elemento) + fim
                    
                if grafo_matriz == 1: elemento += 1
            else:
                    string_matrix += '  ' + fim
        string_matrix += '\n'
        
    string_matrix += ']'

    # Retorna uma string com a matriz:
    return (string_matrix)


# Função que retorna o grafo corresponde da matriz:

def mostra_grafo(grafo):
   
    n = 0
    string_grafo = 'grafo = [\n'
    # Imprime cada linha do grafo:
    for linha in grafo:
        string_grafo += (f'{n}: {linha}\n')
        n += 1
    string_grafo += ']'
        
    return string_grafo

In [7]:
# Programa que mostra os resultados obtidos:
modo = 1
grafo = fazer_grafo(matrix, modo)
tamanhos, caminhos = tamanho_caminhos(grafo)

# Mostra a matriz com uns:
print(mostra_matrix_grafo(matrix, modo, 0))
print('\n')

# Mostra a matriz com os elementos do grafo:
print(mostra_matrix_grafo(matrix, modo, 1))
print('\n')

# Mostra o grafo:
print(mostra_grafo(grafo))
print('\n')

# Mostra cada um dos caminhos e seus tamanhos:
for i in range(len(tamanhos)):
    print(f'Caminho {caminhos[i]} de tamanho {tamanhos[i]}')

matrix = [
[ 1,   ,   ,  1,   ], 
[ 1,   ,  1,   ,   ], 
[  ,   ,  1,   ,  1], 
[ 1,   ,  1,   ,  1], 
[ 1,   ,  1,  1,   ], 
]


matrix = [
[ 0,   ,   ,  1,   ], 
[ 2,   ,  3,   ,   ], 
[  ,   ,  4,   ,  5], 
[ 6,   ,  7,   ,  8], 
[ 9,   , 10, 11,   ], 
]


grafo = [
0: [2]
1: []
2: [0]
3: [4]
4: [3, 7]
5: [8]
6: [9]
7: [4, 10]
8: [5]
9: [6]
10: [7, 11]
11: [10]
]


Caminho [0, 2] de tamanho 2
Caminho [1] de tamanho 1
Caminho [3, 4, 7, 10, 11] de tamanho 5
Caminho [5, 8] de tamanho 2
Caminho [6, 9] de tamanho 2
